In [67]:
import gymnasium as gym
import numpy as np
import random
import matplotlib.pyplot as plt
import math


In [68]:
env=gym.make('MountainCar-v0')
env.reset()
print(env.observation_space)

Box([-1.2  -0.07], [0.6  0.07], (2,), float32)


In [100]:
#setting granulating indices
g_0=100#for car position
g_1=20#for car velocity


In [101]:
def discrete(state):
    r=[0]*2
    #state[0]
    if state[0]<=-1.2:
        r[0]=0
    elif state[0]>=0.6:
        r[0]=g_0
    else:
        for i in range(g_0):
            if state[0]>(-1.2+i*(1.8/g_0)) and state[0]<=(-1.2+(i+1)*(1.8/g_0)):
                r[0]=i+1
    #state[1]
    if state[1]<=-0.07:
        r[1]=0
    elif state[1]>=0.07:
        r[1]=g_1
    else:
        for i in range(g_1):
            if state[1]>(-0.07+i*(0.14/g_1)) and state[1]<=(-0.07+(i+1)*(0.14/g_1)):
                r[1]=i+1
    return r


In [102]:
a=[-1.8,0.05]
print(discrete(a))
act=1
env.step(act)

[0, 18]


(array([0.5142849, 0.012579 ], dtype=float32), -1.0, True, False, {})

In [103]:
g_2=env.action_space.n#3
qtable=np.zeros((g_0+1,g_1+1,g_2))
print (qtable)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [110]:
#parameters
episodes=1000
iterations=10
alpha=0.3
d=1
minimum=-120


In [111]:
avgr = [0] * iterations

for iter in range(iterations):
    qtabletemp = qtable  # Create a copy of the Q-table
    c = 0
    
    for epi in range(episodes):
        state, inf = env.reset()
        done = False
        state = discrete(state)

        ep = 1 / (iter + 1)

        while not done:
            if np.random.uniform(0, 1) < ep:
                action = env.action_space.sample()
            else:
                action = np.argmax(qtable[state[0], state[1]])

            newstate, reward, term, trunc, info = env.step(action)
            c += reward
            newstate = discrete(newstate)

            
            done = trunc or term
            qtabletemp[state[0], state[1], action] = (1 - alpha) * qtabletemp[state[0], state[1], action] + alpha * (reward + d * np.max(qtabletemp[newstate[0], newstate[1]]))
            state = newstate

    c = c / episodes
    avgr[iter] = c

    qtable = qtabletemp  # Update the original Q-table with the values from the temporary copy

print(avgr)


[-200.0, -199.988, -198.831, -195.568, -191.764, -193.85, -189.418, -184.68, -178.903, -184.062]


In [106]:
print(qtable)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [107]:
avgr = [0] * iterations

for iter in range(iterations):
    qtabletemp = qtable  # Create a copy of the Q-table
    c = 0
    
    for epi in range(episodes):
        state, inf = env.reset()
        done = False
        state = discrete(state)

        ep = 1 / (iter+iterations + 1)

        while not done:
            if np.random.uniform(0, 1) < ep:
                action = env.action_space.sample()
            else:
                action = np.argmax(qtable[state[0], state[1]])

            newstate, reward, term, trunc, info = env.step(action)
            c += reward
            newstate = discrete(newstate)

            
            done = trunc or term
            qtabletemp[state[0], state[1], action] = (1 - alpha) * qtabletemp[state[0], state[1], action] + alpha * (reward + d * np.max(qtabletemp[newstate[0], newstate[1]]))
            state = newstate

    c = c / episodes
    avgr[iter] = c

    qtable = qtabletemp  # Update the original Q-table with the values from the temporary copy

print(avgr)


[-197.082, -196.051, -195.886, -194.353, -192.289, -190.814, -194.316, -195.181, -191.611, -190.698]


In [108]:
avgr = [0] * iterations

for iter in range(iterations):
    qtabletemp = qtable  # Create a copy of the Q-table
    c = 0
    
    for epi in range(episodes):
        state, inf = env.reset()
        done = False
        state = discrete(state)

        ep = 1 / (iter+iterations+iterations + 1)

        while not done:
            if np.random.uniform(0, 1) < ep:
                action = env.action_space.sample()
            else:
                action = np.argmax(qtable[state[0], state[1]])

            newstate, reward, term, trunc, info = env.step(action)
            c += reward
            newstate = discrete(newstate)

            
            done = trunc or term
            qtabletemp[state[0], state[1], action] = (1 - alpha) * qtabletemp[state[0], state[1], action] + alpha * (reward + d * np.max(qtabletemp[newstate[0], newstate[1]]))
            state = newstate

    c = c / episodes
    avgr[iter] = c

    qtable = qtabletemp  # Update the original Q-table with the values from the temporary copy

print(avgr)

[-183.907, -183.515, -191.263, -190.535, -187.96, -189.529, -190.863, -190.486, -191.347, -187.816]


In [109]:
avgr = [0] * iterations

for iter in range(iterations):
    qtabletemp = qtable  # Create a copy of the Q-table
    c = 0
    
    for epi in range(episodes):
        state, inf = env.reset()
        done = False
        state = discrete(state)

        ep = 1 / (iter+iterations+iterations + 1)

        while not done:
            if np.random.uniform(0, 1) < ep:
                action = env.action_space.sample()
            else:
                action = np.argmax(qtable[state[0], state[1]])

            newstate, reward, term, trunc, info = env.step(action)
            c += reward
            newstate = discrete(newstate)

            
            done = trunc or term
            qtabletemp[state[0], state[1], action] = (1 - alpha) * qtabletemp[state[0], state[1], action] + alpha * (reward + d * np.max(qtabletemp[newstate[0], newstate[1]]))
            state = newstate

    c = c / episodes
    avgr[iter] = c

    qtable = qtabletemp  # Update the original Q-table with the values from the temporary copy

print(avgr)

[-194.038, -192.78, -190.011, -187.643, -186.788, -191.449, -189.461, -188.939, -190.579, -186.62]
